In [0]:
import time
import typing
import requests


def is_error_response(http_response, seconds_to_sleep: float = 1) -> bool:
    """
    Returns False if status_code is 503 (system unavailable) or 200 (success),
    otherwise it will return True (failed). This function should be used
    after calling the commands requests.post() and requests.get().

    :param http_response:
        The response object returned from requests.post or requests.get.
    :param seconds_to_sleep:
        The sleep time used if the status_code is 503. This is used to not
        overwhelm the service since it is unavailable.
    """
    if http_response.status_code == 503:
        time.sleep(seconds_to_sleep)
        return False

    return http_response.status_code != 200


def get_json(url) -> typing.Union[dict, None]:
    """
    Returns json response if any. Returns None if no json found.

    :param url:
        The url go get the json from.
    """
    response = requests.get(url)
    if is_error_response(response):
        return None
    json_response = response.json()
    return json_response


def get_reviews(app_name, app_id, page=1) -> typing.List[dict]:
    """
    --iOS app reviews--
    
    Returns a list of dictionaries with each dictionary being one review. 
    Maxed out at 500 reviews from itunes server.
    
    :param app_id:
        The app_id you are searching. 
    :param page:
        The page id to start the loop. Once it reaches the final page + 1, the 
        app will not return a 'feed' key in the json, thus it will exit with 
        the current reviews.
    """
    # declare a variable with list of dictionary type
    reviews: typing.List[dict] = [{}]

    while True:
        # requesting json from iOS 
        url = (f'https://itunes.apple.com/rss/customerreviews/id={app_id}/'
               f'page={page}/sortby=mostrecent/json')
        
        json = get_json(url)

        if not json:
            return reviews[1:]

        data_feed = json.get('feed')

        if not data_feed.get("entry"):
            return reviews[1:]

        reviews += [
            {
                'app_name': app_name,
                'app_version': entry.get('im:version').get('label'),
                'label_title': entry.get('title').get('label'),
                'review': entry.get('content').get('label'),
                'store_rating': entry.get('im:rating').get('label')                
            }
            for entry in data_feed.get('entry')
        ]

        page += 1

In [0]:
# List of apps name and ID
app_list = [['rv-life-rv-gps-campgrounds', 1275803975], 
            ['copilot-gps-navigation', 504677517], 
            ['camp-rv-tents-to-rv-parks', 370820516], 
            ['smartrvroute', 606236753], 
            ['rv-parks-campgrounds', 991632449], 
            ['rv-park-finder', 695969271], 
            ['campendium-rv-tent-camping', 1191380095]]

In [0]:
import pandas as pd

df_list = []
for app_name, app_id in app_list:
  app_reviews = get_reviews(app_name, app_id) # get the reviews for specific app
  df = pd.DataFrame(app_reviews) # create dataframe with list of dictionaries
  df_list.append(df) # add into the list of dataframes
  
# merge all dataframes together
df_all = pd.concat(df_list, ignore_index=True, join='outer', axis=0)

In [0]:
print(df_all.shape)
df_all.head()

(1962, 5)


,app_name,app_version,label_title,review,store_rating
0,rv-life-rv-gps-campgrounds,2.2.0,Does nothing,Will not do anything without paid subscription.,1
1,rv-life-rv-gps-campgrounds,2.2.0,Don’t receive confirmation email,Tried several times to obtain a verification e...,1
2,rv-life-rv-gps-campgrounds,2.1.5,This is not ready for release!!! Avoid!!!!,To say we are disappointed is an understatemen...,1
3,rv-life-rv-gps-campgrounds,2.1.5,GPS directions bad,"First time using RV Life, it directed us to a ...",1
4,rv-life-rv-gps-campgrounds,2.1.2,Outstanding ! Essential for campers!,"This is the best, and only app you’ll need to ...",5


In [0]:
# create dataframe into csv
df_all.to_csv("rv_ios_app_reviews.csv", index=False)

In [0]:
import pandas as pd
df_all = pd.read_csv('rv_ios_app_reviews.csv')
df_all['app_name'].value_counts()

copilot-gps-navigation        500
rv-parks-campgrounds          500
camp-rv-tents-to-rv-parks     500
rv-park-finder                252
campendium-rv-tent-camping    116
rv-life-rv-gps-campgrounds     75
smartrvroute                   19
Name: app_name, dtype: int64